In [194]:
from torchvision.datasets import CIFAR10
from torchvision import transforms, models
import torch
from torch import nn
from torch.nn import functional as F
from torch import optim
from torch.utils.data import random_split

In [4]:
train_transform = transforms.Compose([
    transforms.RandomCrop(size=32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])
])

In [5]:
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])
])

In [6]:
train_data = CIFAR10(root='./train/', download=True, train=True, transform=train_transform)

Files already downloaded and verified


In [8]:
test_data = CIFAR10(root='./test/', download=True, train=False, transform=test_transform)

100%|██████████| 170498071/170498071 [03:26<00:00, 824757.41it/s] 


Extracting ./test/cifar-10-python.tar.gz to ./test/


In [10]:
train_data

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./train/
    Split: Train
    StandardTransform
Transform: Compose(
               RandomCrop(size=(32, 32), padding=4)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.201])
           )

In [13]:
#data loader
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=16, shuffle=True)

In [17]:
data_batch, label_batch = next(iter(train_data_loader))
print(data_batch, label_batch)

tensor([[[[ 0.0716, -0.0060, -0.1998,  ..., -0.5293, -0.6069, -0.5681],
          [-0.0641,  0.2848,  0.5174,  ..., -0.6844, -0.6263, -0.5293],
          [ 0.1491,  0.2073,  0.4593,  ..., -0.4518, -0.3355, -0.0060],
          ...,
          [-1.1884, -1.1109, -0.9946,  ..., -0.8201, -0.8395, -0.5487],
          [-0.9946, -0.7038, -0.7813,  ..., -0.8589, -0.9752, -0.8589],
          [-0.8395, -0.7813, -0.7620,  ..., -0.9364, -1.0721, -1.0721]],

         [[ 0.1188, -0.2746, -0.6089,  ..., -0.2746, -0.3336, -0.2942],
          [-0.1172, -0.1566, -0.0976,  ..., -0.5302, -0.4516, -0.3729],
          [-0.0976, -0.0582,  0.0991,  ..., -0.4909, -0.3139, -0.1566],
          ...,
          [-0.6679, -0.5302, -0.5302,  ..., -0.2942, -0.2352,  0.0204],
          [-0.5892, -0.1172, -0.2549,  ..., -0.2549, -0.3139, -0.2746],
          [-0.2942, -0.0976, -0.1566,  ..., -0.3532, -0.3926, -0.4516]],

         [[-0.5045, -0.5825, -0.8947,  ..., -0.8557, -0.9142, -0.8947],
          [-0.6996, -0.4655, -

In [24]:
print(data_batch.size(), label_batch.size())

torch.Size([16, 3, 32, 32]) torch.Size([16])


In [27]:
testloader = torch.utils.data.DataLoader(test_data, batch_size=16, shuffle=False)

In [29]:
test_data

Dataset CIFAR10
    Number of datapoints: 10000
    Root location: ./test/
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.201])
           )

In [31]:
# Model development

#Pretrained models
vgg16 = models.vgg16(pretrained=True)
vgg16

/Users/rakesh.panigrahy/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/rakesh.panigrahy/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /Users/rakesh.panigrahy/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [02:23<00:00, 3.86MB/s] 


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [33]:
#download from hub
# waveglow = torchimport.load('nvidia/DeepLearningExamples:torchhub', 'nvidia_waveglow')

In [76]:
# Build models

class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(2048, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 2)
        
    def forward(self, x):
        x = x.view(-1, 2048)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x), dim=1)
        return x

In [77]:
simplenet = SimpleNet()
simplenet

SimpleNet(
  (fc1): Linear(in_features=2048, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=2, bias=True)
)

In [78]:
input = torch.rand(2048)
input

tensor([0.5984, 0.4306, 0.2755,  ..., 0.5582, 0.3999, 0.5513])

In [79]:
output = simplenet(input)
output

tensor([[0.5671, 0.4329]], grad_fn=<SoftmaxBackward0>)

In [190]:
# implementing LeNet5

class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(3,6,5)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        shape = int(x.nelement()/x.shape[0])
        # print('shape:', shape)
        x = x.view(-1, shape)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [170]:
# conv1 = nn.Conv2d(3,6,5)
# conv2 = nn.Conv2d(6,16,5)
# fc1 = nn.Linear(16*5*5, 120)
# fc2 = nn.Linear(120, 84)
# fc3 = nn.Linear(84, 10)
        
# x = F.max_pool2d(F.relu(conv1(data_batch)), (2,2))
# x = F.max_pool2d(F.relu(conv2(x)), 2)

In [184]:
# shape = x.nelement()/x.shape[0]
# int(shape)

In [185]:
# x.view(-1, 400.0)

In [186]:
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
device

'mps'

In [191]:
model = LeNet5().to(device=device)

In [192]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [193]:
N_EPOCHS = 10

for epoch in range(N_EPOCHS):
    epoch_loss = 0.0
    for inputs, labels in train_data_loader:
        # print(labels, inputs)
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    print('Epoch: {} Loss: {}'.format(epoch, epoch_loss/len(train_data_loader)))

Epoch: 0 Loss: 1.8986050937652588
Epoch: 1 Loss: 1.61238447183609
Epoch: 2 Loss: 1.488969181690216
Epoch: 3 Loss: 1.3959646894454956
Epoch: 4 Loss: 1.3289858430480956
Epoch: 5 Loss: 1.2712296896362305
Epoch: 6 Loss: 1.2296740033531188
Epoch: 7 Loss: 1.1978716300296783
Epoch: 8 Loss: 1.1668466216468811
Epoch: 9 Loss: 1.146901894903183


In [195]:
train_set, val_set = random_split(train_data, [40000, 10000])
train_data_loader = torch.utils.data.DataLoader(train_set, batch_size=16, shuffle=True)
val_data_loader = torch.utils.data.DataLoader(val_set, batch_size=16, shuffle=True)
print(len(train_data_loader), len(val_data_loader))

2500 625


In [196]:
model = LeNet5().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), momentum=0.9, lr=0.001)

In [197]:
N_EPOCHS = 10

for epoch in range(N_EPOCHS):
    
    # Training
    train_loss = 0.0
    model.train()
    for inputs, labels in train_data_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        
        loss = criterion(outputs, labels)
        
        loss.backward()
        
        optimizer.step()
        
        train_loss += loss.item()
        
    # Validation
    val_loss = 0.0
    model.eval()
    for inputs, labels in val_data_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = model(inputs)
        
        loss = criterion(outputs, labels)
        
        val_loss += loss.item()
        
    print('Epoch: {} Train loss: {} Val loss: {}'.format(epoch, train_loss/len(train_data_loader), val_loss/len(val_data_loader)))

Epoch: 0 Train loss: 1.986956847667694 Val loss: 1.8022550579071044
Epoch: 1 Train loss: 1.6551996953487396 Val loss: 1.60591105260849
Epoch: 2 Train loss: 1.5284061108589173 Val loss: 1.4320575778961182
Epoch: 3 Train loss: 1.427760422563553 Val loss: 1.4083028846740722
Epoch: 4 Train loss: 1.3572310300827026 Val loss: 1.3267935526847838
Epoch: 5 Train loss: 1.3019365445375441 Val loss: 1.3271397072792053
Epoch: 6 Train loss: 1.2568526196956635 Val loss: 1.244089520263672
Epoch: 7 Train loss: 1.2250370062708855 Val loss: 1.2774292875766755
Epoch: 8 Train loss: 1.1939902986884117 Val loss: 1.153288900232315
Epoch: 9 Train loss: 1.1727744385361671 Val loss: 1.186674709033966


In [199]:
#Testing

num_correct = 0.0
for x_test_batch, y_test_batch in testloader:
    model.eval()
    y_test_batch = y_test_batch.to(device)
    x_test_batch = x_test_batch.to(device)
    
    y_pred_batch = model(x_test_batch)
    
    _, predicted = torch.max(y_pred_batch, 1)
    
    num_correct += (predicted == y_test_batch).float().sum()
    
accuracy = num_correct/(len(testloader)*testloader.batch_size)

print(len(testloader), testloader.batch_size)
print('Test accuracy: {}'.format(accuracy))

625 16
Test accuracy: 0.6159999966621399


In [201]:
# Model deployment

torch.save(model, './len5_model.pt')
torch.save(model.state_dict(), './len5_model_weights.pt')

In [ ]:
m1 = torch.load('./len5_model.pt')
m1

LeNet5(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [217]:
m2 = LeNet5().to(device)
m2.load_state_dict(torch.load('./len5_model_weights.pt'))
m2

LeNet5(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)